In [ ]:
import pandas as pd
from sqlalchemy.orm import Session

import src
import src.db.connect
import src.db.models.bert_data as bm

In [ ]:
engine = src.db.connect.make_engine("DB")
s = Session(engine)

In [ ]:
valid_coders = ["grabsch", "schadt", "coudry", "riedel", "richter"]

results = []
for coder in valid_coders:
    query = s.query(bm.Sample).filter(
        bm.Sample.used_in_batch != None,
        ~s.query(bm.Label)
        .filter(bm.Sample.id == bm.Label.sample_id, bm.Label.username == coder)
        .exists(),
    )
    with engine.connect() as conn:
        df = pd.read_sql(query.statement, conn)

    results.append({"coder": coder, "data": df})

In [ ]:
[(d["coder"], len(d["data"])) for d in results]

[('grabsch', 0), ('schadt', 0), ('coudry', 0), ('riedel', 0), ('richter', 0)]

In [ ]:
tmpdir = src.PATH / "tmp"
tmpdir.mkdir(exist_ok=True)


for data in results:
    coder = data["coder"]
    df = data["data"]
    df["label"] = ""

    with open(tmpdir / f"missings_{coder}.jsonl", "w", encoding="utf-8") as file:
        df[["text", "label", "id"]].to_json(file, orient="records", lines=True, force_ascii=False)